In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, explode, split, regexp_replace, lower

In [2]:
SONG_LYRICS_FILES = "/home/jovyan/work/data/song_lyrics/*.txt"

In [3]:
spark = (
    SparkSession.builder
    .master("spark://spark-master:7077")
    .appName("word-count")
    .getOrCreate()
)

In [4]:
song_lyrics_df = (
    spark.read
    .format("text")
    .load(SONG_LYRICS_FILES)
)

In [5]:
words_df = (
    song_lyrics_df
    .withColumn("word", explode(split(col("value"), "\s+")))
    .withColumn("word", regexp_replace(col("word"), "^\W+|\W+$", ""))
    .where(col("word") != "")
)

In [6]:
counted_unique_words_df = (
    words_df
    .groupBy("word")
    .count()
    .orderBy(col("count").desc())
)

In [7]:
print("total_words")
words_df.count()

total_words


1345811

In [8]:
print("word_counts")
counted_unique_words_df.show(10, False)

word_counts
+----+-----+
|word|count|
+----+-----+
|I   |46120|
|the |44602|
|you |34876|
|to  |24623|
|a   |24263|
|me  |20285|
|my  |17297|
|and |17248|
|it  |15984|
|in  |15287|
+----+-----+
only showing top 10 rows



In [9]:
spark.stop()